# 实时疫情数据

数据来源：腾讯新闻

In [ ]:
import requests
import json
import pandas as pd

url = r'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
response = requests.get(url)
result = json.loads(response.json()['data'])
data_china = result['areaTree'][0]['children']

# item['total']总数据，替换为item['today']可获得当日新增数据
def GetTotalChina(data_china):
    province_list=[]
    city_list=[]
    confirm_list=[]
    dead_list=[]
    suspect_list=[]
    heal_list=[]
    
    for each in data_china:
        for item in each['children']:
            province_list.append(each['name'])
            city_list.append(item['name'])
            confirm_list.append(item['total']['confirm'])
            suspect_list.append(item['total']['suspect'])
            dead_list.append(item['total']['dead'])
            heal_list.append(item['total']['heal'])

    ddict = {'province':province_list,
            'city':city_list,
            'confirm':confirm_list,
            'suspect':suspect_list,
            'dead':dead_list,
            'heal':heal_list}
    df = pd.DataFrame(ddict, columns=['province','city','confirm','suspect','dead', 'heal'])
    confirm_prov = df.groupby(['province'])['confirm'].sum().reset_index()
    confirm_prov = confirm_prov.sort_values(by=['confirm'], ascending=False)
    confirm_prov = confirm_prov.reset_index(drop=True)
    
    return df

def GetTodayChina(data_china):
    province_list=[]
    city_list=[]
    confirm_list=[]
    dead_list=[]
    suspect_list=[]
    heal_list=[]
    
    for each in data_china:
        for item in each['children']:
            province_list.append(each['name'])
            city_list.append(item['name'])
            confirm_list.append(item['today']['confirm'])
            suspect_list.append(item['today']['suspect'])
            dead_list.append(item['today']['dead'])
            heal_list.append(item['today']['heal'])

    ddict = {'province':province_list,
            'city':city_list,
            'confirm':confirm_list,
            'suspect':suspect_list,
            'dead':dead_list,
            'heal':heal_list}
    df = pd.DataFrame(ddict, columns=['province','city','confirm','suspect','dead', 'heal'])
    confirm_prov = df.groupby(['province'])['confirm'].sum().reset_index()
    confirm_prov = confirm_prov.sort_values(by=['confirm'], ascending=False)
    confirm_prov = confirm_prov.reset_index(drop=True)
    
    return df

dftotal = GetTotalChina(data_china)

# 绘制确诊病例数分布图（直辖市、省、自治区级）

In [1]:
from pyecharts import options as opts
from pyecharts.charts import Map

def DrawMapChina(data):
    
    confirm_prov = data.groupby(['province'])['confirm'].sum().reset_index()
    confirm_prov = confirm_prov.sort_values(by=['confirm'], ascending=False)
    confirm_prov = confirm_prov.reset_index(drop=True)

    province = confirm_prov['province'].values.tolist()
    confirm = confirm_prov['confirm'].values.tolist()

    geo = Map()
    geo.add("2019-nCoV确诊人数", [list(z) for z in zip(province, confirm)], "china")
    geo.set_global_opts(visualmap_opts=opts.VisualMapOpts(max_=300),
                       title_opts=opts.TitleOpts(title="2019-nCoV"))
    geo.render('map.html')
    return geo.render_notebook()
    
DrawMapChina(dftotal)

# 绘制确诊病例热力图（直辖市、省、自治区级）

In [4]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType

confirm_prov = dftotal.groupby(['province'])['confirm'].sum().reset_index()
confirm_prov = confirm_prov.sort_values(by=['confirm'], ascending=False)
confirm_prov = confirm_prov.reset_index(drop=True)

province = confirm_prov['province'].values.tolist()
confirm = confirm_prov['confirm'].values.tolist()

c = (
    Geo()
    .add_schema(maptype="china")
    .add(
        "确诊人数",
        [list(z) for z in zip(province, confirm)],
        type_=ChartType.HEATMAP,
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(max_=200),
        title_opts=opts.TitleOpts(title="Geo-HeatMap 2019-nCoV"),
    )
)

c.render_notebook()